### Imports

In [13]:
import os
import pandas as pd
import numpy as np

### Parameters

In [14]:
num_samples = 50 # quantidade de exemplos

### Inputs

In [15]:
def generate_realistic_edge(is_good_link):
    if is_good_link:
        rssi_mean = -55
        rssi_std = 8
        rssi = np.random.normal(rssi_mean, rssi_std)
        rssi = np.clip(rssi, -85, -30)
        
        etx = 1 + np.random.exponential(0.8)
        etx = np.clip(etx, 1, 8)
        
        delay = np.random.lognormal(2.3, 0.6)
        delay = np.clip(delay, 1, 80)
        
        busy = np.random.beta(2, 5)
        busy = np.clip(busy, 0.05, 0.7)
        
    else:
        rssi_mean = -75
        rssi_std = 10
        rssi = np.random.normal(rssi_mean, rssi_std)
        rssi = np.clip(rssi, -95, -45)
        
        etx = 1 + np.random.gamma(3, 1.2)
        etx = np.clip(etx, 1.5, 15)
        
        delay = np.random.lognormal(3.5, 0.8)
        delay = np.clip(delay, 10, 150)
        
        busy = np.random.beta(5, 2)
        busy = np.clip(busy, 0.1, 1.0)
    
    rssi += np.random.uniform(-0.05, 0.05)
    etx += np.random.uniform(-0.01, 0.01)
    delay += np.random.uniform(-0.2, 0.2)
    busy += np.random.uniform(-0.005, 0.005)
    
    rssi = np.clip(rssi, -95, -30)
    etx = np.clip(etx, 1.0, 15)
    delay = np.clip(delay, 1, 150)
    busy = np.clip(busy, 0.0, 1.0)
    
    if rssi < -70:
        etx *= np.random.uniform(1.1, 1.4)
        delay *= np.random.uniform(1.05, 1.25)
        etx = min(etx, 15)
    
    if busy > 0.6:
        delay *= np.random.uniform(1.2, 1.6)
        etx *= np.random.uniform(1.05, 1.2)
        delay = min(delay, 150)
        etx = min(etx, 15)
    
    if etx > 7:
        delay *= np.random.uniform(1.1, 1.3)
        delay = min(delay, 150)
    
    rssi = round(rssi, 2)
    etx = round(etx, 3)
    delay = round(delay, 2)
    busy = round(busy, 4)
    
    return rssi, etx, delay, busy

edge_inputs = []
label_0_count = 0
label_1_count = 0
target_per_label = num_samples // 2

while len(edge_inputs) < num_samples:
    if label_0_count >= target_per_label:
        target_label = 1
    elif label_1_count >= target_per_label:
        target_label = 0
    else:
        target_label = 1 if np.random.random() < 0.52 else 0
    
    is_good = (target_label == 1)
    
    rssi, etx, delay, busy = generate_realistic_edge(is_good)
    
    if target_label == 0:
        label_0_count += 1
    else:
        label_1_count += 1
    
    edge_inputs.append([rssi, etx, delay, busy])

In [16]:
def generate_realistic_radnet_metrics(is_good_performance):
    if is_good_performance:
        apmr_global = np.random.beta(9, 2)
        apmr_global = 0.75 + (apmr_global * 0.20)
        
        mfsr_global = np.random.beta(10, 2)
        mfsr_global = 0.80 + (mfsr_global * 0.16)
        
        idg_global = np.random.beta(8, 3)
        idg_global = 0.68 + (idg_global * 0.24)
        
    else:
        apmr_global = np.random.beta(3, 6)
        apmr_global = 0.45 + (apmr_global * 0.32)
        
        mfsr_global = np.random.beta(3, 5)
        mfsr_global = 0.48 + (mfsr_global * 0.35)
        
        idg_global = np.random.beta(2, 6)
        idg_global = 0.42 + (idg_global * 0.32)
    
    apmr_global += np.random.uniform(-0.01, 0.01)
    mfsr_global += np.random.uniform(-0.01, 0.01)
    idg_global += np.random.uniform(-0.01, 0.01)
    
    apmr_global = np.clip(apmr_global, 0.40, 0.98)
    mfsr_global = np.clip(mfsr_global, 0.45, 0.98)
    idg_global = np.clip(idg_global, 0.40, 0.95)
    
    if mfsr_global < apmr_global:
        mfsr_global = apmr_global * np.random.uniform(1.0, 1.08)
        mfsr_global = min(mfsr_global, 0.98)
    
    if apmr_global < 0.55:
        mfsr_global *= np.random.uniform(0.85, 0.95)
        idg_global *= np.random.uniform(0.88, 0.96)
    
    if mfsr_global < 0.60:
        idg_global *= np.random.uniform(0.82, 0.93)
    
    apmr_global = round(apmr_global, 6)
    mfsr_global = round(mfsr_global, 6)
    idg_global = round(idg_global, 4)
    
    return apmr_global, mfsr_global, idg_global

radnet_inputs = []
good_performance_count = 0
poor_performance_count = 0
target_per_performance = num_samples // 2

while len(radnet_inputs) < num_samples:
    if good_performance_count >= target_per_performance:
        target_performance = 0
    elif poor_performance_count >= target_per_performance:
        target_performance = 1
    else:
        target_performance = 1 if np.random.random() < 0.52 else 0
    
    is_good = (target_performance == 1)
    apmr_g, mfsr_g, idg_g = generate_realistic_radnet_metrics(is_good)
    
    if target_performance == 0:
        poor_performance_count += 1
    else:
        good_performance_count += 1
    
    radnet_inputs.append([apmr_g, mfsr_g, idg_g])

### Save

In [17]:
edges_input = pd.DataFrame(edge_inputs, columns=["rssi", "etx", "delay", "busy_fraction"])
os.makedirs("../inputs", exist_ok=True)
edges_input.to_csv("../inputs/edges_input.csv", index=False)

print("Edges Inputs salvo em ../datasets/edges_input.csv")

Edges Inputs salvo em ../datasets/edges_input.csv


In [18]:
radnet_input = pd.DataFrame(radnet_inputs, columns=["apmr_edge", "mfsr_edge", "idg_global"])
os.makedirs("../inputs", exist_ok=True)
radnet_input.to_csv("../inputs/radnet_input.csv", index=False)

print("Radnet Input salvo em ../datasets/radnet_input.csv")

Radnet Input salvo em ../datasets/radnet_input.csv
